# Estimation of building height using digital elevation data

This work strives to test a hypothesis. The hypothesis is that a building height can be estimated by means of the difference between digital terrain model (DTM) and digital surface model (DSM). To test the hypothesis, city of Halle in Germany is considerd in combination with openly available data from Saxony-Anhalt (https://www.lvermgeo.sachsen-anhalt.de/de/kostenfreie_geobasisdaten_lvermgeo.html) and Open Street Map (https://www.openstreetmap.org). 

The following procedure is used to test the hypothesis. In the __first__ part, DSM, DTM, and geometry of the buildings are extracted. In the __second__ part, DSM and DTM data are intersected on building geometry and the differenced between the DSM and DTM heights are calculated. And in the __third__ part, a weighted average method is used to estimate flat height with confidence interval of buildings. 

In [1]:
import geopandas as gpd
import pandas as pd
import osmnx as ox
from shapely.geometry import LineString, Point, Polygon

import os 

In [2]:
# Toggle between users

# Babak
upath = r'C:\Users\beb\OneDrive - NILU'

## ''Part 1''

#### Reading and concatination of DTM and DSM files

In [3]:
%%time
# get list of DTM files in the following directory
fd_dtm = os.path.join(upath, r'doc\FAIRiCUBE\WP2\data\DTM')
dtmfiles = []

for fn in os.listdir(fd_dtm):
    if os.path.isfile(os.path.join(fd_dtm, fn)):
        dtmfiles.append(os.path.join(fd_dtm,fn))


# get list of DSM files in the following directory
fd_dsm = os.path.join(upath, r'doc\FAIRiCUBE\WP2\data\DSM')
dsmfiles = []

for fn in os.listdir(fd_dsm):
    if os.path.isfile(os.path.join(fd_dsm, fn)):
        dsmfiles.append(os.path.join(fd_dsm,fn))

CPU times: total: 0 ns
Wall time: 9.97 ms


In [4]:
%%time
# extractin .xyz files asspacited with the DTM and DSM data and convert them to geo-files and later concat them.

# geodataframe containg DTM and DSM
gdtm = gpd.GeoDataFrame(columns = ['x','y','z','geometry'], geometry = 'geometry')
gdsm = gpd.GeoDataFrame(columns = ['x','y','z','geometry'], geometry = 'geometry')

for dtm in dtmfiles: #dtmfiles[:1]
    print('Index number of the read DTM file = ' + str(dtmfiles.index(dtm)))
    df = []
    df = pd.read_table(dtm, delim_whitespace=True, names=['x', 'y', 'z'])
    gdtm = pd.concat([gdtm,
                      gpd.GeoDataFrame(df, 
                                       crs = 'epsg:25832', 
                                       geometry = gpd.points_from_xy(df.x,df.y))], 
                     ignore_index=True)

for dsm in dsmfiles:
    print('Index number of the read DSM file = ' + str(dsmfiles.index(dsm)))
    df = []
    df = pd.read_table(dsm, delim_whitespace=True, names=['x', 'y', 'z'])
    gdsm = pd.concat([gdsm,
                      gpd.GeoDataFrame(df, 
                                       crs = 'epsg:25832', 
                                       geometry = gpd.points_from_xy(df.x,df.y))], 
                     ignore_index=True)

# .buffer(1,cap_style = 3)

Index number of the read DTM file = 0
Index number of the read DTM file = 1
Index number of the read DTM file = 2
Index number of the read DTM file = 3
Index number of the read DTM file = 4
Index number of the read DTM file = 5
Index number of the read DTM file = 6
Index number of the read DTM file = 7
Index number of the read DTM file = 8
Index number of the read DTM file = 9
Index number of the read DTM file = 10
Index number of the read DTM file = 11
Index number of the read DTM file = 12
Index number of the read DTM file = 13
Index number of the read DTM file = 14
Index number of the read DTM file = 15
Index number of the read DTM file = 16
Index number of the read DTM file = 17
Index number of the read DTM file = 18
Index number of the read DTM file = 19
Index number of the read DTM file = 20
Index number of the read DTM file = 21
Index number of the read DTM file = 22
Index number of the read DTM file = 23
Index number of the read DTM file = 24
Index number of the read DTM file =

#### Extraction of building geometries from openstreetmap

In [5]:
%%time
# Setting a boundary box for the city of Halle in Germany
south, east, north, west = [51.3617,12.2793,51.5837,11.6651]

# Extracting data associated with building
buildings = ox.geometries_from_bbox(north, south, east, west, tags = {'building': True})

# attributes of interest associated with buildings
bcols = ['geometry', 'building', 'building:levels']
bdata = buildings[bcols]

# extracting only polygon geometries for buildigns
bdata = bdata[bdata['geometry'].geom_type == 'Polygon']

CPU times: total: 35.3 s
Wall time: 36 s


#### Carrying out a series of GIS processing

In [6]:
%%time
# droping DTM and DSM points that are not overlaying with building geometry
# converting decimal degree to meter via ".to_crs(epsg=3763)"
for i in ['gdtm','gdsm']:
    locals()['b_'+i] = gpd.overlay(bdata.to_crs(epsg=25832).reset_index(),
                                   locals()[i].set_crs(epsg=25832), 
                                   how='intersection',
                                   keep_geom_type=False)

# buffering (2-by-2 meter) remaining points from the previous transformer that are within the geometry of buildings
for i in ['b_gdtm','b_gdsm']:
    dum = []
    dum = locals()[i].copy()
    
    del dum['geometry']
    dum['geometry'] = Polygon([(0, 0), (0, 0), (0, 0)])
    dum['geometry'] = locals()[i].buffer(1,cap_style = 3)
    locals()['b'+i] = dum

# filltering out buffered points that are not 100% within the building geometry
for i in ['bb_gdtm','bb_gdsm']:
    
    dum = []
    dum = gpd.overlay(bdata.to_crs(epsg=25832).reset_index(),
                      locals()[i][['x','y','z','geometry']],
                      how='intersection',
                      keep_geom_type=False)
    
    locals()['bbw'+i[-5:]] = dum[dum.area == 4]


# intersecting DTM and DSM
dem = gpd.overlay(bbw_gdtm[['osmid','building','building:levels','x','y','z','geometry']],
                  bbw_gdsm[['z','geometry']],
                  how='intersection',
                  keep_geom_type=False)

# calculating the difference 
dem['z_meter'] = (dem['z_2']-dem['z_1']).astype(float)

<timed exec>:15: FutureWarning: Setting geometries to a GeoDataFrame without a geometry column will currently preserve the CRS, if present. This is deprecated, and in the future the CRS will be lost in this case. You can use set_crs(..) on the result to set the CRS manually.
<timed exec>:15: FutureWarning: Setting geometries to a GeoDataFrame without a geometry column will currently preserve the CRS, if present. This is deprecated, and in the future the CRS will be lost in this case. You can use set_crs(..) on the result to set the CRS manually.


CPU times: total: 1h 57min 36s
Wall time: 2h 11min 26s


#### Estimation of building height
In this approach, the created DEM file is grouped by building's 'OSM ID's and then mean, median, min, and max are calculated.

In [8]:
save_dir = 'doc\FAIRiCUBE\WP2\data\DEM_halle'

# dissolving the created "dem" dataframe by DTM x and y 
dem_diss_b = dem.dissolve(by=['osmid', 'building'], 
                        aggfunc = {'z_meter': 'mean'
                                  })

# save as shapefile
dem_diss_b.to_file(os.path.join(upath , save_dir, 'dem_diss_b.shp'))

<>:1: DeprecationWarning: invalid escape sequence '\F'
<>:1: DeprecationWarning: invalid escape sequence '\F'
C:\Users\beb\AppData\Local\Temp\ipykernel_19584\4135358725.py:1: DeprecationWarning: invalid escape sequence '\F'
  save_dir = 'doc\FAIRiCUBE\WP2\data\DEM_halle'


#### CUBE representation of DEM  (creating a raster file using DTM as the reference geometry)
In this approach, the created DEM file is grouped by DTM's 'x' and 'y' coordinates and later the mean of heights associated with DTM and DSM are calculated. NB. here, CUBEs that are not 2-by-2 are filtered out.

In [9]:
# dissolving the created "dem" dataframe by DTM x and y 
dem_diss_c = dem.dissolve(by=['x','y'], 
                          aggfunc = {'osmid': 'mean',
                                     'building': 'first',
                                     'z_1': 'mean',
                                     'z_2': 'mean',
                                     'z_meter': 'mean'
                                    })
# rename some columns
dem_diss_c = dem_diss_c.rename(columns = {'z_1':'z_terrain',
                                          'z_2': 'z_surface'})



# save as shapefile
dem_diss_c.to_file(os.path.join(upath , save_dir, 'dem_diss_c.shp'))